In [1]:
import pandas as pd

# Load and Preprocess Data

In [2]:
stations = pd.read_csv("Divvy_Bicycle_Stations_20240722.csv")
trips = pd.read_csv("Divvy_Trips_20240721.csv")

## Preprocess

Format time to Pandas datetime format.

In [3]:
trips['START TIME'] = pd.to_datetime(trips['START TIME'], format='%m/%d/%Y %I:%M:%S %p')
trips['STOP TIME'] = pd.to_datetime(trips['STOP TIME'], format='%m/%d/%Y %I:%M:%S %p')

To expedite query speed, we calculate the prefix sum `NET INFLOW` and `NET OUTFLOW` for every `STATION ID` at every `TIME`, so the query just needs to look up the first hits for two conditions.

In [4]:
outflows = trips[['START TIME', 'FROM STATION ID']].copy()
outflows['OUTFLOW'] = 1
outflows.rename(columns={'START TIME': 'TIME', 'FROM STATION ID': 'STATION ID'}, inplace=True)

inflows = trips[['STOP TIME', 'TO STATION ID']].copy()
inflows['INFLOW'] = 1
inflows.rename(columns={'STOP TIME': 'TIME', 'TO STATION ID': 'STATION ID'}, inplace=True)

events = pd.merge(inflows, outflows, on=['TIME', 'STATION ID'], how='outer')
events['INFLOW'] = events['INFLOW'].fillna(0)
events['OUTFLOW'] = events['OUTFLOW'].fillna(0)

events['INFLOW'] = events['INFLOW'].astype(int)
events['OUTFLOW'] = events['OUTFLOW'].astype(int)

events.sort_values(by=['STATION ID', 'TIME'], inplace=True)

events['NET INFLOW'] = events.groupby('STATION ID')['INFLOW'].cumsum()
events['NET OUTFLOW'] = events.groupby('STATION ID')['OUTFLOW'].cumsum()
events['NET FLOW'] = events['NET INFLOW'] - events['NET OUTFLOW']

In [5]:
def query_flow(station_id, start_time, stop_time):
    start_time = pd.to_datetime(start_time)
    stop_time = pd.to_datetime(stop_time)
    
    filtered_before = events[(events['STATION ID'] == station_id) & (events['TIME'] <= start_time)]
    if filtered_before.empty:
        start_inflow = 0
        start_outflow = 0
    else:
        start_inflow = filtered_before.iloc[-1]["NET INFLOW"]
        start_outflow = filtered_before.iloc[-1]["NET OUTFLOW"]
    
    filtered_after = events[(events['STATION ID'] == station_id) & (events['TIME'] >= stop_time)]
    if filtered_after.empty:
        stop_inflow = events[events['STATION ID'] == station_id].iloc[-1]["NET INFLOW"]
        stop_outflow = events[events['STATION ID'] == station_id].iloc[-1]["NET OUTFLOW"]
    else:
        stop_inflow = filtered_after.iloc[0]["NET INFLOW"]
        stop_outflow = filtered_after.iloc[0]["NET OUTFLOW"]
    
    net_inflow = stop_inflow - start_inflow
    net_outflow = stop_outflow - start_outflow
    net_flow = net_inflow - net_outflow
    
    return net_inflow, net_outflow, net_flow

Use daily statistics to determine if a station is overflow (more days with bikes returned `1`), balanced (no difference `0`), or underflow (more days with bikes taken `-1`), and calculate the net overflow days and net underflow days for each station.

In [6]:
daily = events[["STATION ID", "TIME", "INFLOW", "OUTFLOW"]].copy()
daily['DATE'] = daily['TIME'].dt.date
daily['STATION ID, DATE'] = daily['STATION ID'].astype(str) + ', ' + daily['DATE'].astype(str)
daily = daily.groupby('STATION ID, DATE').agg({"STATION ID": "first", "DATE": "first", "INFLOW": "sum", "OUTFLOW": "sum"})
daily["DAILY FLOW"] = daily["INFLOW"] - daily["OUTFLOW"]
daily = daily.reset_index()[["STATION ID", "DATE", "INFLOW", "OUTFLOW", "DAILY FLOW"]]
daily["STATUS"] = daily["DAILY FLOW"].apply(lambda x: 1 if x > 0 else -1 if x < 0 else 0).astype(int)

In [7]:
station_results = daily.groupby("STATION ID").agg({"STATUS": "sum"})
station_results["RESULT"] = station_results["STATUS"].apply(lambda x: "OVERFLOW" if x > 0 else "UNDERFLOW" if x < 0 else "BALANCED")

## Demo

In [8]:
events[(events['STATION ID'] == 2) & (events['TIME'] <= "2015-05-10 0:00:00")].tail()

,TIME,STATION ID,INFLOW,OUTFLOW,NET INFLOW,NET OUTFLOW,NET FLOW
7105013,2015-05-09 18:42:00,2,0,1,55,61,-6
7105054,2015-05-09 18:46:00,2,0,1,55,62,-7
7105083,2015-05-09 18:50:00,2,0,1,55,63,-8
7106053,2015-05-09 20:44:00,2,0,1,55,64,-9
7106191,2015-05-09 21:16:00,2,1,0,56,64,-8


In [9]:
query_flow(2, "2015-05-09", "2015-05-10")  # returned: net_inflow, net_outflow, net_flow

(45, 50, -5)

In [11]:
events.head()

,TIME,STATION ID,INFLOW,OUTFLOW,NET INFLOW,NET OUTFLOW,NET FLOW
35565237,2019-05-21 14:01:05,1,0,1,0,1,-1
35565302,2019-05-21 14:10:20,1,1,0,1,1,0
7087349,2015-05-08 12:23:00,2,1,0,1,0,1
7087350,2015-05-08 12:23:00,2,1,0,2,0,2
7087351,2015-05-08 12:23:00,2,1,0,3,0,3


In [12]:
daily.head()

,STATION ID,DATE,INFLOW,OUTFLOW,DAILY FLOW,STATUS
0,1,2019-05-21,1,1,0,0
1,100,2013-07-17,12,12,0,0
2,100,2013-07-18,18,19,-1,-1
3,100,2013-07-19,19,16,3,1
4,100,2013-07-20,20,24,-4,-1


In [13]:
trips.head()

,TRIP ID,START TIME,STOP TIME,BIKE ID,TRIP DURATION,FROM STATION ID,FROM STATION NAME,TO STATION ID,TO STATION NAME,USER TYPE,GENDER,BIRTH YEAR,FROM LATITUDE,FROM LONGITUDE,FROM LOCATION,TO LATITUDE,TO LONGITUDE,TO LOCATION
0,8546790,2015-12-31 17:35:00,2015-12-31 17:44:00,979,521,117,Wilton Ave & Belmont Ave,229,Southport Ave & Roscoe St,Subscriber,Female,1991.0,41.940180,-87.653040,POINT (-87.65304 41.94018),41.943739,-87.664020,POINT (-87.66402 41.943739)
1,8546793,2015-12-31 17:37:00,2015-12-31 17:41:00,1932,256,301,Clark St & Schiller St,138,Clybourn Ave & Division St,Subscriber,Male,1992.0,41.907993,-87.631501,POINT (-87.631501 41.907993),41.904613,-87.640552,POINT (-87.640552 41.904613)
2,8546795,2015-12-31 17:37:00,2015-12-31 17:40:00,1693,134,465,Marine Dr & Ainslie St,251,Clarendon Ave & Leland Ave,Subscriber,Female,1987.0,41.971600,-87.650154,POINT (-87.650154 41.9716),41.967968,-87.650001,POINT (-87.650001 41.967968)
3,8546797,2015-12-31 17:38:00,2015-12-31 17:55:00,3370,995,333,Ashland Ave & Blackhawk St,198,Green St (Halsted St) & Madison St,Subscriber,Male,1975.0,41.907066,-87.667252,POINT (-87.667252 41.907066),41.881892,-87.648789,POINT (-87.648789 41.881892)
4,8546798,2015-12-31 17:38:00,2015-12-31 17:41:00,2563,177,48,Larrabee St & Kingsbury St,111,Sedgwick St & Huron St,Subscriber,Male,1990.0,41.897764,-87.642884,POINT (-87.642884 41.897764),41.894666,-87.638437,POINT (-87.638437 41.894666)


In [14]:
station_results.head()

,STATUS,RESULT
STATION ID,,
1,0,BALANCED
2,-589,UNDERFLOW
3,-825,UNDERFLOW
4,13,OVERFLOW
5,237,OVERFLOW


## Save Processed Data

In [15]:
events.to_parquet("events.parquet")
daily.to_parquet("daily.parquet")
station_results.to_csv("station_results.csv")